In [1]:
%matplotlib inline
import numpy as np
from copy import deepcopy

from scipy.misc import derivative
import scipy as spy
from scipy import interpolate
import matplotlib
#matplotlib.use('TkAgg', force=True)
import matplotlib.pyplot as plt
import math
import cPickle as pickle
import datetime
#from IPython.display import Image as img
#from IPython.display import Markdown as md
#from IPython.display import display as dp
import string as st

from scipy.misc import imread

from __future__ import division

from fatiando import gravmag, mesher, utils, gridder
from fatiando.mesher import Prism, Polygon
from fatiando.gravmag import prism
from fatiando.utils import ang2vec, si2nt, contaminate
from fatiando.gridder import regular
from fatiando.vis import mpl

from numpy.testing import assert_almost_equal
from numpy.testing import assert_array_almost_equal
from pytest import raises

plt.rc('font', size=16)

/Users/macelabastos/anaconda2/lib/python2.7/site-packages/fatiando/vis/mpl.py:76: UserWarning: This module will be removed in v0.6. We recommend the use of matplotlib.pyplot module directly. Some of the fatiando specific functions will remain.
  "specific functions will remain.")


In [2]:
import functions as fc

In [3]:
# Model`s limits
ymin = 0.0
ymax = 150000.0
zmin = -1000.0
zmax = 45000.0
xmin = -100000.0
xmax = 100000.0
area = [ymin, ymax, zmax, zmin]
# number of observation datas and number of prisms along the profile
ny = 10

# coordinates defining the horizontal boundaries of the
# adjacent columns along the profile
y = np.linspace(ymin, ymax, ny)

# coordinates of the center of the columns forming the
# interpretation model
n = ny - 1
dy = (ymax - ymin)/n
ycmin = ymin + 0.5*dy
ycmax = ymax - 0.5*dy
yc = np.reshape(np.linspace(ycmin, ycmax, n),(n,1))
x = np.zeros_like(yc)
z = np.zeros_like(yc)-150.0

In [4]:
# Model densities
dw = np.array([1030.0])
ds0 = np.array([2400.0])
dsig = np.array([2800.0])
dcc = np.array([2670.0])
doc = np.array([2840.0])
dm = np.array([3300.0])
#dc = dcc
dc = np.zeros_like(yc)
COT = 80000.0
aux = yc <= COT
for i in range(len(yc[aux])):
    dc[i] = dcc
for i in range(len(yc[aux]),n):
    dc[i] = doc

# defining sediments layers density vector
ds = np.vstack((np.reshape(np.repeat(ds0,n),(1,n)),np.reshape(np.repeat(dsig,n),(1,n))))

# S0 => isostatic compensation surface (Airy's model)
# SR = S0+dS0 => reference Moho (Forward modeling)
S0 = np.array([45000.0])
dS0 = np.array([8300.0])

In [5]:
assert np.alltrue(dw < ds), 'Sediments densities without geological meaning'
assert np.alltrue(ds0 < dc), 'Crust densities without geological meaning'
assert np.alltrue(dsig > dcc), 'Crust densities without geological meaning'
assert np.alltrue(dsig < doc), 'Crust densities without geological meaning'
assert np.alltrue(dc < dm), 'Mantle densities without geological meaning'

In [6]:
# parameter vector limits
#pjmin = np.array([-50.0 - 10**(-10)]) 
#pjmax = np.array([100000.0 + 10**(-10)])
# finite difference step
dp1 = 1.
dp2 = 1000.

In [7]:
yc

array([[   8333.33333333],
       [  25000.        ],
       [  41666.66666667],
       [  58333.33333333],
       [  75000.        ],
       [  91666.66666667],
       [ 108333.33333333],
       [ 125000.        ],
       [ 141666.66666667]])

In [8]:
## Edge extension (observation coordinates)
sigma = 0.5
edge = sigma*dy*n

In [9]:
#print yc

In [10]:
bath_picks = np.array([[    453.62903226,    1846.77419355],
       [   2570.56451613,    2451.61290323],
       [  17086.69354839,    3207.66129032],
       [  40070.56451613,    3812.5       ],
       [  41885.08064516,    3358.87096774],
       [  44304.43548387,    3661.29032258],
       [  54889.11290323,    3812.5       ],
       [  56098.79032258,    3510.08064516],
       [  57762.09677419,    3661.29032258],
       [  94354.83870968,    3812.5       ],
       [  98286.29032258,    4114.91935484],
       [ 100100.80645161,    4266.12903226],
       [ 148790.32258065,    4266.12903226]])

bath_picks[0,0] = ymin
bath_picks[-1,0] = ymax
bathymetry = fc.surface_interpolate_function(bath_picks,yc)
tw = np.reshape(bathymetry,(n,1))

In [11]:
toi_picks = np.array([[    705.64516129,    1266.38104839],
       [  30342.74193548,    1619.20362903],
       [  51864.91935484,    2677.67137097],
       [  60685.48387097,    4088.96169355],
       [  70211.69354839,    4441.78427419],
       [  76209.67741935,    5147.42943548],
       [  82560.48387097,    4794.60685484],
       [  85030.24193548,    5147.42943548],
       [  88558.46774194,    5500.25201613],
       [  90322.58064516,    4441.78427419],
       [ 110080.64516129,    4794.60685484],
       [ 123487.90322581,    6558.71975806],
       [ 132661.29032258,    7617.1875    ],
       [ 147832.66129032,    6911.54233871],
       [ 160534.27419355,    6911.54233871],
       [ 163709.67741935,    6205.89717742],
       [ 167590.72580645,    6558.71975806],
       [ 178175.40322581,    6558.71975806],
       [ 202167.33870968,    6911.54233871],
       [ 222631.0483871 ,    6911.54233871],
       [ 253679.43548387,    7617.1875    ],
       [ 271320.56451613,    6205.89717742],
       [ 277671.37096774,    6558.71975806],
       [ 287903.22580645,    6911.54233871],
       [ 299193.5483871 ,    6558.71975806],
       [ 305191.53225806,    6911.54233871],
       [ 348588.70967742,    6558.71975806]])

toi_picks[0,0] = ymin
toi_picks[-1,0] = ymax

toi = fc.surface_interpolate_function(toi_picks,yc)
for i in range(len(toi)):
    if toi[i] < bathymetry[i]:
        toi[i] = bathymetry[i]

ts0 = toi - bathymetry

In [12]:
basement_picks = np.array([[    705.64516129,    2324.84879032],
       [   5997.98387097,    1972.02620968],
       [   9879.03225806,    3030.49395161],
       [  13760.08064516,    2324.84879032],
       [  17993.9516129 ,    3736.1391129 ],
       [  25403.22580645,    9381.30040323],
       [  31754.03225806,   16790.57459677],
       [  39516.12903226,   23847.02620968],
       [  47631.0483871 ,   27375.25201613],
       [  55040.32258065,   27728.07459677],
       [  69506.0483871 ,   24199.84879032],
       [  74445.56451613,   25963.96169355],
       [  77620.96774194,   26316.78427419],
       [  82207.66129032,   25611.1391129 ],
       [  91381.0483871 ,   18201.86491935],
       [  95614.91935484,   19613.15524194],
       [ 100201.61290323,   19613.15524194],
       [ 105846.77419355,   18554.6875    ],
       [ 111491.93548387,   16790.57459677],
       [ 118901.20967742,   11498.2358871 ],
       [ 122782.25806452,   12909.52620968],
       [ 125604.83870968,   12203.88104839],
       [ 135483.87096774,   13615.17137097],
       [ 139717.74193548,   12556.70362903],
       [ 151360.88709677,   13967.99395161],
       [ 155947.58064516,   12556.70362903],
       [ 171471.77419355,   13967.99395161],
       [ 185937.5       ,   15026.46169355],
       [ 189465.72580645,   16437.75201613],
       [ 197933.46774194,   16790.57459677],
       [ 203225.80645161,   14673.6391129 ],
       [ 210282.25806452,   15732.10685484],
       [ 213810.48387097,   15379.28427419],
       [ 216985.88709677,   16437.75201613],
       [ 227923.38709677,   13967.99395161],
       [ 233215.72580645,   14320.81653226],
       [ 239566.53225806,   13615.17137097],
       [ 243800.40322581,   15379.28427419],
       [ 249092.74193548,   16084.92943548],
       [ 258618.9516129 ,   12556.70362903],
       [ 263558.46774194,   14320.81653226],
       [ 271320.56451613,   16084.92943548],
       [ 285080.64516129,   16084.92943548],
       [ 297429.43548387,   13967.99395161],
       [ 302721.77419355,   11498.2358871 ],
       [ 304485.88709677,    9734.12298387],
       [ 348235.88709677,    9381.30040323]])

basement_picks[0,0] = ymin
basement_picks[-1,0] = ymax

basement = fc.surface_interpolate_function(basement_picks,yc)
for i in range(len(basement)):
    if basement[i] < toi[i]:
        basement[i] = toi[i]

ts1 = basement - toi

In [13]:
ts = np.vstack((np.reshape(ts0,(1,n)),np.reshape(ts1,(1,n))))

In [14]:
S = fc.moho_function(S0,dw,ds,dcc,dm,dc,tw,ts)
tm = S0 - S
toc = S - tw - ts0 - ts1

In [15]:
print '[', np.reshape(yc,(n,))[1], ',' , np.reshape(basement,(n,))[1], '],'
print '[', np.reshape(yc,(n,))[5], ',' , np.reshape(basement,(n,))[5], '],'
print '[', np.reshape(yc,(n,))[8], ',' , np.reshape(basement,(n,))[8], ']'

print '[', np.reshape(yc,(n,))[2], ',' , np.reshape(S,(n,))[2], '],'
print '[', np.reshape(yc,(n,))[7], ',' , np.reshape(S,(n,))[7], ']'


[ 25000.0 , 9074.08074117 ],
[ 91666.6666667 , 18297.0710125 ],
[ 141666.666667 , 11954.829815 ]
[ 41666.6666667 , 40523.5170413 ],
[ 125000.0 , 41995.4839384 ]


In [16]:
base_known = np.array([[ 25000.0 , 9074.08074117 ],
[ 91666.6666667 , 18297.0710125 ],
[ 141666.666667 , 11954.829815 ]])
moho_known = np.array([[ 41666.6666667 , 40523.5170413 ],
[ 125000.0 , 41995.4839384 ]])

In [17]:
assert np.alltrue(tw <= toi), 'top of igneous layer must be deeper than the seabed'

In [18]:
assert np.alltrue(toi <= basement), 'basement must be deeper than the top of igneous layer'

In [19]:
assert np.alltrue(basement <= S), 'Moho must be deeper than the basement'

In [20]:
assert np.alltrue(S <= S0), 'Compensation depth must be placed below the Moho'

In [21]:
p = []
p = np.vstack((ts1, tm, dS0))
#print p.size

In [22]:
prism_w = fc.prism_w_function(xmax,xmin,dy,edge,dw,dcc,tw,yc)
prism_s = fc.prism_s_function(xmax,xmin,dy,edge,ds,dcc,tw,p,yc,ts0,two_layers=True)
prism_c = fc.prism_c_function(xmax,xmin,dy,edge,S0,dcc,dc,tw,p,yc,ts0,two_layers=True)
prism_m = fc.prism_m_function(xmax,xmin,dy,edge,S0,dcc,dm,p,yc)

In [23]:
gzw = prism.gz(np.reshape(x,(n,)),np.reshape(yc,(n,)),np.reshape(z,(n,)),prism_w)
gzs0 = prism.gz(np.reshape(x,(n,)),np.reshape(yc,(n,)),np.reshape(z,(n,)),prism_s[1])
gzs1 = prism.gz(np.reshape(x,(n,)),np.reshape(yc,(n,)),np.reshape(z,(n,)),prism_s[2])
dcaux = np.copy(dc)
for i in range(n):
    dc[i] = dcc
prism_caux = fc.prism_c_function(xmax,xmin,dy,edge,S0,dcc,dc,tw,p,yc,ts0,two_layers=True)
gzc = prism.gz(np.reshape(x,(n,)),np.reshape(yc,(n,)),np.reshape(z,(n,)),prism_caux)
gzm = prism.gz(np.reshape(x,(n,)),np.reshape(yc,(n,)),np.reshape(z,(n,)),prism_m)

#Calculo do vetor de dados observados:
g = fc.g_function(np.reshape(x,(n,)),np.reshape(yc,(n,)),np.reshape(z,(n,)),gzw,prism_s,prism_caux,prism_m)

In [24]:
f = 2.34565

pdens_w = f*(dw-dcc)
pdens_s0 = f*(ds[0,0]-dcc)
pdens_s1 = f*(ds[1,0]-dcc)
pdens_c = f*(dc[0]-dcc)
pdens_m = f*(dm-dcc)

gzw2 = prism.gz(np.reshape(x,(n,)),np.reshape(yc,(n,)),np.reshape(z,(n,)),prism_w, dens=pdens_w)
gzs02 = prism.gz(np.reshape(x,(n,)),np.reshape(yc,(n,)),np.reshape(z,(n,)),prism_s[1], dens=pdens_s0)
gzs12 = prism.gz(np.reshape(x,(n,)),np.reshape(yc,(n,)),np.reshape(z,(n,)),prism_s[2], dens=pdens_s1)
gzc2 = prism.gz(np.reshape(x,(n,)),np.reshape(yc,(n,)),np.reshape(z,(n,)),prism_caux, dens=pdens_c)
gzm2 = prism.gz(np.reshape(x,(n,)),np.reshape(yc,(n,)),np.reshape(z,(n,)),prism_m, dens=pdens_m)
    
g2 = gzw2 + gzs02 + gzs12 + gzc2 + gzm2

dc = np.copy(dcaux)

In [25]:
assert_almost_equal(gzw2, f*gzw, decimal=9)

In [26]:
assert_almost_equal(gzs02, f*gzs0, decimal=9)

In [27]:
assert_almost_equal(gzs12, f*gzs1, decimal=9)

In [28]:
assert_almost_equal(gzc2, f*gzc, decimal=9)

In [29]:
assert_almost_equal(gzm2, f*gzm, decimal=9)

In [30]:
assert_almost_equal(g2, f*g, decimal=9)

In [31]:
raises(ValueError, prism.gz, x[:-2], yc, z, prism_w)
raises(ValueError, prism.gz, x, yc[:-2], z, prism_w)
raises(ValueError, prism.gz, x, yc, z[:-2], prism_w)
raises(ValueError, prism.gz, x[:-5], yc, z[:-2], prism_w)


<ExceptionInfo ValueError tblen=2>

In [32]:
#parameter_vector = fc.parameter_vector(S0, prism_w, prism_s, prism_c, prism_m)
parameter_vector = fc.parameter_vector(S0,prism_w,prism_s,prism_c,prism_m,two_layers=True)

assert_almost_equal(p, parameter_vector, decimal=11)

In [33]:
(rs,index_rs) = fc.base_known_function(dy,tw,yc,base_known,ts0,two_layers=True)
(rm,index_rm) = fc.moho_known_function(dy,yc,S0,moho_known)

In [34]:
index_base = index_rs
index_moho = index_rm - n

In [35]:
assert_almost_equal(base_known[:,0], yc[index_base][:,0], decimal=6)

assert_almost_equal(moho_known[:,0], yc[index_moho][:,0], decimal=6)

assert_almost_equal(ts1[index_base][:,0], rs[:,0], decimal=6)

assert_almost_equal((tm[index_moho][:,0]), rm[:,0], decimal=6)

In [36]:
t = np.array([4., 6., 8.]).reshape((3,1))
v = np.array([1.,2.,3.]).reshape((3,1))

tmin = np.reshape(np.repeat(2.,len(t)),(len(t),1))
tmax = np.reshape(np.repeat(10.,len(t)),(len(t),1))

T1 = fc.T_matrix_function(tmin,tmax,t)
v1 = T1.dot(v)

T2 = np.array([[1.5, 0, 0],
             [0, 2, 0],
             [0, 0, 1.5]])
v2 = T2.dot(v)

assert_array_almost_equal(v1, v2, decimal=11)

In [37]:
nr=3
R01 = np.array([[1., -1., 0.],
               [0., 1., -1.]])

R02 = fc.R_matrix_function(nr)

assert_array_almost_equal(R01, R02, decimal=11)

In [38]:
dcaux = np.copy(dc[3:6])
dsaux = np.copy(ds[:,3:6])

C1 = fc.C_matrix_function(dsaux,dm,dcaux,two_layers=True)

I = np.identity(len(dcaux))

C2 = np.hstack([(dsaux[1,:] - dcaux)*I, (dm - dcaux)*I, np.zeros((len(dcaux),1))])

assert_array_almost_equal(C1, C2, decimal=11)
print C1
print C2

[[ 130.    0.    0.  630.    0.    0.    0.]
 [   0.  130.    0.    0.  630.    0.    0.]
 [   0.    0.  -40.    0.    0.  460.    0.]]
[[ 130.    0.    0.  630.    0.    0.    0.]
 [   0.  130.    0.    0.  630.    0.    0.]
 [  -0.   -0.  -40.    0.    0.  460.    0.]]


In [39]:
# iso
R0 = fc.R_matrix_function(n)
dwiso = np.array([1030.0])
ds0iso = np.array([2350.0])
ds1iso = np.array([2750.0])
dcciso = np.array([2770.0])
dociso = np.array([2865.0])
dmiso = np.array([3300.0])
#dc = dcc
# list defining crust density variance
dciso = np.zeros_like(yc)
aux = yc <= COT
for i in range(len(yc[aux])):
    dciso[i] = dcciso
for i in range(len(yc[aux]),n):
    dciso[i] = dociso    
# defining sediments layers density matrix
dsiso = np.vstack((np.reshape(np.repeat(ds0iso,n),(1,n)),np.reshape(np.repeat(ds1iso,n),(1,n))))

Ciso = fc.C_matrix_function(dsiso,dmiso,dciso,two_layers=True)

#print Ciso

Hess_iso = 2*Ciso.T.dot(R0.T.dot(R0.dot(Ciso)))
diag_iso = np.diag(Hess_iso)
f_iso = np.median(diag_iso)

#print Hess_iso
print diag_iso
print f_iso

[  8.00000000e+02   1.60000000e+03   1.60000000e+03   1.60000000e+03
   1.60000000e+03   5.29000000e+04   5.29000000e+04   5.29000000e+04
   2.64500000e+04   5.61800000e+05   1.12360000e+06   1.12360000e+06
   1.12360000e+06   1.12360000e+06   7.56900000e+05   7.56900000e+05
   7.56900000e+05   3.78450000e+05   0.00000000e+00]
52900.0


In [40]:
# noiso

dwniso = np.array([1030.0])
ds0niso = np.array([2350.0])
ds1niso = np.array([2855.0])
dccniso = np.array([2870.0])
docniso = np.array([2885.0])
dmniso = np.array([3230.0])
# list defining crust density variance
dcniso = np.zeros_like(yc)
aux = yc <= COT
for i in range(len(yc[aux])):
    dcniso[i] = dccniso
for i in range(len(yc[aux]),n):
    dcniso[i] = docniso    
# defining sediments layers density matrix
dsniso = np.vstack((np.reshape(np.repeat(ds0niso,n),(1,n)),np.reshape(np.repeat(ds1niso,n),(1,n))))

Cniso = fc.C_matrix_function(dsniso,dmniso,dcniso,two_layers=True)

#print Cniso

Hess_niso = 2*Cniso.T.dot(R0.T.dot(R0.dot(Cniso)))
diag_niso = np.diag(Hess_niso)
f_niso = np.median(diag_niso)

#print Hess_niso
print diag_niso
print f_niso


[  4.50000000e+02   9.00000000e+02   9.00000000e+02   9.00000000e+02
   9.00000000e+02   3.60000000e+03   3.60000000e+03   3.60000000e+03
   1.80000000e+03   2.59200000e+05   5.18400000e+05   5.18400000e+05
   5.18400000e+05   5.18400000e+05   4.76100000e+05   4.76100000e+05
   4.76100000e+05   2.38050000e+05   0.00000000e+00]
3600.0


In [41]:
D1 = fc.D_matrix_function(dw,dcaux,dsaux,two_layers=True)

D2 = np.hstack([(dw - dcaux)*I, (dsaux[0,:] - dcaux)*I, dcaux])

assert_array_almost_equal(D1, D2, decimal=11)
print D1
print D2

[[-1640.     0.     0.  -270.     0.     0.  2670.]
 [    0. -1640.     0.     0.  -270.     0.  2670.]
 [    0.     0. -1810.     0.     0.  -440.  2840.]]
[[-1640.    -0.    -0.  -270.    -0.    -0.  2670.]
 [   -0. -1640.    -0.    -0.  -270.    -0.  2670.]
 [   -0.    -0. -1810.    -0.    -0.  -440.  2840.]]


In [42]:
ps = np.array([1., 2., 3., 4., 6., 8., 10.]).reshape((7,1))
nps = int((len(ps)-1)/2)
R = fc.R_matrix_function(nps)
Sa1 = np.array([[1, -1, 0, 0, 0, 0, 0],
               [0, 1, -1, 0, 0, 0, 0]])
print Sa1
Sa2 = fc.Sa_matrix_function(nps)
print Sa2
assert_array_almost_equal(Sa1, Sa2, decimal=11)

[[ 1 -1  0  0  0  0  0]
 [ 0  1 -1  0  0  0  0]]
[[ 1. -1.  0.  0.  0.  0.  0.]
 [ 0.  1. -1.  0.  0.  0.  0.]]


In [43]:
Sb1 = np.array([[0, 0, 0, 1, -1, 0, 0],
               [0, 0, 0, 0, 1, -1, 0]])
print Sb1
Sb2 = fc.Sb_matrix_function(nps)
print Sb2
assert_array_almost_equal(Sb1, Sb2, decimal=11)

[[ 0  0  0  1 -1  0  0]
 [ 0  0  0  0  1 -1  0]]
[[ 0.  0.  0.  1. -1.  0.  0.]
 [ 0.  0.  0.  0.  1. -1.  0.]]


In [44]:
Sap = Sa2.dot(ps)
print Sap
Sbp = Sb2.dot(ps)
print Sbp

[[-1.]
 [-1.]]
[[-2.]
 [-2.]]


In [45]:
(rs1,index_rs1) = fc.base_known_function(dy,tw,yc,base_known,ts0,two_layers=True)
A1 = fc.A_matrix_function(n,rs1,index_rs1)

index_rs2 = []
indices = np.arange(2*n+1, dtype=int)
for bi in range(len(rs1)):
    mask = A1[bi,:] == 1.
    index_rs2.append(indices[mask])    
index_rs2 = (np.array(index_rs2)).reshape(len(rs1))

assert_array_almost_equal(index_rs1, index_rs2, decimal=11)

In [46]:
(rm1,index_rm1) = fc.moho_known_function(dy,yc,S0,moho_known)
B1 = fc.B_matrix_function(n,rm1,index_rm1)

index_rm2 = []
indices = np.arange(2*n+1, dtype=int)
for mi in range(len(rm1)):
    mask = B1[mi,:] == 1.
    index_rm2.append(indices[mask])    
index_rm2 = (np.array(index_rm2)).reshape(len(rm1))

assert_array_almost_equal(index_rm1, index_rm2, decimal=11)

In [47]:
W = fc.W_matrix_function(1000.,np.reshape(g,(n,1)),np.reshape(g2,(n,1)))
C = fc.C_matrix_function(ds,dm,dc,two_layers=True)
D = fc.D_matrix_function(dw,dc,ds,two_layers=True)
R = fc.R_matrix_function(n)
Sa = fc.Sa_matrix_function(n)
Sb = fc.Sb_matrix_function(n)
t = np.vstack((tw, ts0, S0))

In [57]:
gama1 = fc.gama_function(1.,1.,1.,1.,1.,1.,S0,tw,np.reshape(g,(n,1)),np.reshape(g2,(n,1)),p,rs1,rm1,W,R,C,D,Sa,Sb,A1,B1,ts0,two_layers=True)

gama2 = (1./n)*((g - g2).T.dot(g - g2)) + 1.*((1.)*((W.dot(R.dot(D.dot(t))) + W.dot(R.dot(C.dot(p)))).T.dot(W.dot(R.dot(D.dot(t))) + W.dot(R.dot(C.dot(p)))))[0,0] +
                                             (1.)*((Sa.dot(p)).T.dot(Sa.dot(p)))[0,0] + 
                                             (1.)*((Sb.dot(p)).T.dot(Sb.dot(p)))[0,0] + 
                                             (1.)*((A1.dot(p) - rs1).T.dot((A1.dot(p) - rs1)))[0,0] + 
                                             (1.)*((B1.dot(p) - rm1).T.dot((B1.dot(p) - rm1)))[0,0])
print gama1, gama2
assert_array_almost_equal(gama1, gama2, decimal=15)

444470528.835 444470528.835


In [49]:
grd_psi0_1 = fc.grad_psi_iso_function(S0,tw,p,W,R,C,D,ts0,two_layers=True)

grd_psi0_2 = 2.*(((((C.T.dot(R.T)).dot(W.T)).dot(W)).dot(R)).dot((D.dot(t) + C.dot(p))))

assert_array_almost_equal(grd_psi0_1, grd_psi0_2, decimal=15)

In [50]:
grd_psi1_1 = fc.grad_psi_tk1_function(p,Sa)

grd_psi1_2 = 2.*(Sa.T.dot(Sa)).dot(p)

assert_array_almost_equal(grd_psi1_1, grd_psi1_2, decimal=15)

In [51]:
grd_psi2_1 = fc.grad_psi_tk1_function(p,Sb)

grd_psi2_2 = 2.*(Sb.T.dot(Sb)).dot(p)

assert_array_almost_equal(grd_psi2_1, grd_psi2_2, decimal=15)

In [52]:
grd_psi3_1 = fc.grad_psi_eq_function(p,rs1,A1)

grd_psi3_2 = 2.*A1.T.dot(A1.dot(p)-rs1)

assert_array_almost_equal(grd_psi3_1, grd_psi3_2, decimal=15)

In [53]:
grd_psi4_1 = fc.grad_psi_eq_function(p,rm1,B1)

grd_psi4_2 = 2.*B1.T.dot(B1.dot(p)-rm1)

assert_array_almost_equal(grd_psi4_1, grd_psi4_2, decimal=15)

In [54]:
print len(yc)

9


In [55]:
G1 = fc.G_matrix_function(xmax,xmin,dy,edge,dp1,dp2,S0,dw,ds,dm,dcc,dc,tw,p,yc,ts0,two_layers=True)
#G2 = fc.G_matrix_function_all(xmax,xmin,dy,edge,dp1,dp2,COT,S0,dw,ds,dm,dcc,dc,tw,p,yc,area,3)
#assert_array_almost_equal(G1, G2, decimal=3)

#a funcao G2 foi modificada e so funciona com uma camada de sedimento

## Teste do modelo direto usando funcao analitica